In [ ]:
"""
実行結果を解析するスクリプトです。
"""
import pandas as pd

data = pd.read_csv("../data/exchange_data/step3_glove.csv")

data = data[~data["result"].str.contains("empty|keyerror")]
data.to_csv("../data/exchange_data/step3_result.csv")
print(len(data))

In [1]:
"""
コーパスの確認をするスクリプトです。
"""
import re
import pandas as pd

clean_text = re.compile('[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％]')

data = pd.read_csv("../data/train_data/recipe_memo.csv",names=["step"])
cleaned_data = data.apply(lambda data:data.str.replace(clean_text,'',regex=True))

In [2]:
cleaned_data = cleaned_data.dropna(subset=["step"])
cleaned_data["str_len"] = cleaned_data["step"].apply(lambda data:len(data))
output = cleaned_data[cleaned_data["str_len"] != 0]
output.describe()

,str_len
count,1.081126e+07
mean,2.822550e+01
std,4.860047e+02
min,1.000000e+00
25%,1.600000e+01
50%,2.500000e+01
75%,3.800000e+01
max,1.334866e+06


In [3]:
output.describe()
#output["step"].to_csv('../data/train_data/fix_recipe_memo.csv')
print(len(output))

10811257


In [29]:
output["str_len"].sum()

305153110

In [6]:
"""
コーパスの作成
"""
from tqdm import tqdm as tq
import MeCab

m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -Owakati")

# 合計語彙数
num = 0
# データの総文字数
str_len = output["str_len"].sum()
# 学習データ
corpas = ""

for step in tq(output["step"],total=len(output)):
    wakati = m.parse(step)
    num += len(wakati.split(' '))
    corpas += wakati

with open("../data/train_data/cookpad_step_corpas.txt",'w') as out_corpas:
    out_corpas.write(corpas)

print("平均語彙サイズ:",str_len/num)
print("平均語彙数:",num/len(output))

100%|██████████| 10811257/10811257 [07:16<00:00, 24759.61it/s]


平均語彙サイズ: 1.796835761781823
平均語彙数: 15.708446113157795


In [2]:
import MeCab

m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -Owakati")

unko = m.parse("うんこたべたいです")
print(unko.split(' '))

['うんこ', 'たべ', 'たい', 'です', '\n']
